In [21]:
import sys; sys.path.append("../..")
import blame
import binvec
import binfunc
import ranking
import cgm
from bidict import bidict

In [22]:
# Substitution rank conformity -- examples
# hash: arksMPOe2p

def check_is_substitute(f11,f10,f01,f00):
    print(f"f00 = {f00} \t\t f01 ={f01}")
    print(f"f10 = {f10} \t\t f11 ={f11}")
    print("Holds:" if f00^f01 >= f00^f10 else "Does not hold:", f"f00^f01 = {f00^f01} >= {f00^f10} = f00^f10")
    print("Holds:" if f11^f10 >= f11^f01 else "Does not hold:", f"f11^f10 = {f11^f10} >= {f11^f01} = f11^f01")

print()
print("Example #1")
print("-"*10)
f00 = binfunc.from_lambda(lambda z: 0)
f01 = binfunc.from_lambda(lambda z: 1)
f10 = binfunc.from_lambda(lambda z: z)
f11 = binfunc.from_lambda(lambda z: 1)
check_is_substitute(f11,f10,f01,f00)

print()
print("Example #2")
print("-"*10)
f00 = binfunc.from_lambda(lambda z: 0)
f01 = binfunc.from_lambda(lambda z: z)
f10 = binfunc.from_lambda(lambda z: z)
f11 = binfunc.from_lambda(lambda z: ~z)
check_is_substitute(f11,f10,f01,f00)

print()
print("Example #3")
print("-"*10)
f00 = binfunc.from_lambda(lambda z1,z2,z3: 0)
f01 = binfunc.from_lambda(lambda z1,z2,z3: z1)
f10 = binfunc.from_lambda(lambda z1,z2,z3: z2&z3)
f11 = binfunc.from_lambda(lambda z1,z2,z3: z1|z2&z3)
check_is_substitute(f11,f10,f01,f00)


Example #1
----------
f00 = 0 		 f01 =1
f10 = z 		 f11 =1
Holds: f00^f01 = 1 >= z = f00^f10
Holds: f11^f10 = z' >= 0 = f11^f01

Example #2
----------
f00 = 0 		 f01 =z
f10 = z 		 f11 =z'
Holds: f00^f01 = z >= z = f00^f10
Holds: f11^f10 = 1 >= 1 = f11^f01

Example #3
----------
f00 = 0 		 f01 =z1
f10 = z2z3 		 f11 =z2z3 + z1
Does not hold: f00^f01 = z1 >= z2z3 = f00^f10
Does not hold: f11^f10 = z1z2' + z1z3' >= z1'z2z3 = f11^f01


In [23]:
# Blame: Different choices of rho
# hash: 4viwaDwcCi
def f(x0,x1,x2,x3,x4):
    return ~x0&~x3&~x1&~x4 | x0&(~x3&~x1&x4 | ~x3&x1&~x4 | x2&~x1 | x2&~x4 | x3&x2) | ~x3&x2&~x4
f = binfunc.from_lambda(f)

r1 = ranking.from_val(lambda x,f: blame.blame(x,f,rho=lambda x:1/(x+1)), f)
r2 = ranking.from_val(lambda x,f: blame.blame(x,f,rho=lambda x: 2**-x),  f)
print("rho(x) = 1/(x+1):\t", r1)
print("rho(x) = 2^(-x):\t", r2)

rho(x) = 1/(x+1):	 x2:0.5260 < x1:0.5323 < x4:0.6182 < x3:0.6432 < x0:0.7135
rho(x) = 2^(-x):	 x1:0.4863 < x2:0.5234 < x4:0.5840 < x3:0.6211 < x0:0.6953


In [24]:
# Blame: Example for switching function
# hash: 6BWW7D7Yqq
h = binfunc.from_lambda(lambda x1,x2: ~x1&x2)
g = binfunc.from_lambda(lambda y1,y2:  y1&y2)
pi, S = bidict({"x1": "y1", "x2": "y2"}), {"x1"}
assert g == h.flip(S).rename(pi)

u = binvec.vec({"y1": 1, "y2": 1, "x1": 1, "x2": 1, "z1": 0})
uhat = u.apply_switch(pi, S)
assert uhat == binvec.vec({"y1": 0, "y2": 1, "x1": 0, "x2": 1, "z1": 0})

In [25]:
# Blame: Example for SRC 
# hash: GZmiukBjJ4
h = binfunc.from_lambda(lambda x1,x2: x1|x2)
g = binfunc.from_lambda(lambda y1,y2: y1|y2)
f = g & (h | binfunc.from_lambda(lambda z1: z1))

u = binvec.vec({ "y1": 0, "y2": 0, "x1": 0, "x2": 1, "z1": 1 })
uhat = binvec.vec({ "y1": 0, "y2": 1, "x1": 0, "x2": 0, "z1": 1 })
u_op_x1 = binvec.vec({ "y1": 0, "y2": 0, "x1": 1, "x2": 1, "z1": 1 })
uhat_op_y1 = binvec.vec({ "y1": 1, "y2": 1, "x1": 0, "x2": 0, "z1": 1 })

print("r^(u)_x1(f) and r^(u_op_x1)_x1(f):\t\t", blame.r("x1", u, f), blame.r("x1", u_op_x1, f))
print("r^(uhat)_y1(f) and r^(uhat_op_x1)_y1(f):\t", blame.r("y1", uhat, f), blame.r("y1", uhat_op_y1, f))

r^(u)_x1(f) and r^(u_op_x1)_x1(f):		 3 inf
r^(uhat)_y1(f) and r^(uhat_op_x1)_y1(f):	 inf 1


In [26]:
# Blame: Counterexample for Property NIM^S
# hash: tsXirYjENE 
h = binfunc.from_lambda(lambda x,y: x|y)
f = h ^ binfunc.from_lambda(lambda z: z)
print("phi^rho_x(h):\t", blame.blame("x", h) )
print("phi^rho_x(f):\t", blame.blame("x", f) )

phi^rho_x(h):	 0.625
phi^rho_x(f):	 0.7083333333333334


In [27]:
# Blame: Counterexample for Property MRP^W
# hash: 0ZfZS5iLCY
h = binfunc.from_lambda(lambda x0,x1,x2,x3: x1&(~x0&x2 | x3) | x0&x2&x3)
f = h | binfunc.from_lambda(lambda z0: z0)
b = lambda x,f: blame.blame(x,f,rho=lambda x:1/(x+1))
print(ranking.from_val(b, h))
print(ranking.from_val(b, f))

x2:0.4375 < x0:0.5521 < x1:0.6250 == x3:0.6250
x2:0.3125 < x1:0.4167 == x3:0.4167 < x0:0.4292 < z0:0.7188


In [28]:
# Bz/DCGM and Bz/RCGM: non-compliance to MRP^S, CI, NIM^S
# hash: OyLAay4541
f = binfunc.from_lambda(lambda x,y,z: x&(y^z))
bz_dcgm = cgm.get_value("Bz/DCGM")
bz_rcgm = cgm.get_value("Bz/RCGM")
print("Bz^omega_x(f) | Bz^omega_x(~f) | Bz^nu_x(f) | Bz^nu_x(~f):\t", 
    bz_dcgm("x", f), "|", bz_dcgm("x", ~f), "|", bz_rcgm("x", f), "|", bz_rcgm("x", ~f))
print("Bz^omega_y(f) | Bz^omega_y(~f) | Bz^nu_y(f) | Bz^nu_y(~f):\t", 
    bz_dcgm("y", f), "|", bz_dcgm("y", ~f), "|", bz_rcgm("y", f), "|", bz_rcgm("y", ~f))

Bz^omega_x(f) | Bz^omega_x(~f) | Bz^nu_x(f) | Bz^nu_x(~f):	 0.25 | 0.75 | 0.75 | 0.25
Bz^omega_y(f) | Bz^omega_y(~f) | Bz^nu_y(f) | Bz^nu_y(~f):	 0.25 | 0.25 | 0.25 | 0.25


In [29]:
# Bz/DCGM and Bz/RCGM: non-compliance to SRC^S
# hash: fKwFA86xIm
g = binfunc.from_lambda(lambda y1,y2: y1&y2)
h = binfunc.from_lambda(lambda x1,x2: x1&x2)
f00 = binfunc.from_lambda(lambda z1,z2: z1^z2)
f01 = binfunc.from_lambda(lambda z1,z2: ~(z1^z2))
f10 = binfunc.from_lambda(lambda z1,z2: 0)
f11 = binfunc.from_lambda(lambda z1,z2: ~z1|z2)
f = h&g&f11 | h&~g&f10 | ~h&g&f01 | ~h&~g&f00
assert f00^f10 <= f00^f01
assert f11^f01 <= f11^f10

bz_dcgm = cgm.get_value("Bz/DCGM")
bz_rcgm = cgm.get_value("Bz/RCGM")
print("Bz^nu_x1(f) =", bz_rcgm("x1", f), ">", bz_rcgm("y1", f), "= Bz^nu_y1(f)")
print("Bz^omega_x1(~f) =", bz_dcgm("x1", ~f), ">", bz_rcgm("y1", ~f), "= Bz^omega_y1(~f)")

Bz^nu_x1(f) = 0.3125 > 0.1875 = Bz^nu_y1(f)
Bz^omega_x1(~f) = 0.3125 > 0.1875 = Bz^omega_y1(~f)


In [30]:
# Bz/RCGM symmetrization
# hash: ewufMPvadu
h = binfunc.from_lambda(lambda x,y,z1: ~x&z1 | x&~z1 | x&y | y&z1)
f = h & binfunc.from_lambda(lambda z2: z2)
bz_srcgm = cgm.get_value("Bz/SRCGM")
print("Bz^(tilde nu)_x(h) vs. Bz^(tilde nu)_y(h):\t", bz_srcgm("x", h), bz_srcgm("y", h))
print("Bz^(tilde nu)_x(f) vs. Bz^(tilde nu)_y(f):\t", bz_srcgm("x", f), bz_srcgm("y", f))

Bz^(tilde nu)_x(h) vs. Bz^(tilde nu)_y(h):	 0.5 0.5
Bz^(tilde nu)_x(f) vs. Bz^(tilde nu)_y(f):	 0.25 0.0


In [31]:
# Bz/QHCGM counterexample SRC^S
# hash: cXobaNe2jw
f00 = binfunc.from_lambda(lambda z0,z1,z2: ~z1|~z2|z0)
f01 = binfunc.from_lambda(lambda z0,z1,z2: z2&(z0|z1))
f10 = binfunc.from_lambda(lambda z0,z1,z2: z2&z0)
f11 = binfunc.from_lambda(lambda z0,z1,z2: z1&~z0)
assert f00^f01 >= f00^f10
assert f11^f10 >= f11^f01
print(f"f00^f10 = {f00^f10} <= {f00^f01} = f00^f01")
print(f"f11^f01 = {f11^f01} <= {f11^f10} = f11^f10")

g = binfunc.from_lambda(lambda y1,y2: y1&y2)
h = binfunc.from_lambda(lambda x1,x2: x1&x2)
f = h&g&f11 | h&~g&f10 | ~h&g&f01 | ~h&~g&f00

v = cgm.get_value("Bz/QH")
print("Bz^E_x1(f) = ", v("x1", f), ">", v("y1", f), " = Bz^E_y1(f)")

f00^f10 = z1'z0' + z2' <= z2' + z0' = f00^f01
f11^f01 = z1z2'z0' + z2z0 <= z2z0 + z1z0' + z1z2 = f11^f10
Bz^E_x1(f) =  0.1260223388671875 > 0.1172332763671875  = Bz^E_y1(f)
